In [35]:
#from tifffile import imsave, imread

#%%GPU memory prepare
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
 
#from tifffile import imsave, imread

#%%GPU memory prepare
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
 
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.80
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
import os
import numpy as np


# Imports (unchanged)
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import os
import numpy as np

# Function to load images and masks
def load_img(img_dir, img_list, mask_dir, mask_list):
  # Create a dictionary to map image names to corresponding mask names
  image_mask_dict = {}
  for image_name, mask_name in zip(img_list, mask_list):
    image_mask_dict[image_name] = mask_name

  images = []
  masks = []
  for image_name in img_list:
    if (image_name.split('.')[1] == 'npy'):
      # Load image
      image = np.load(os.path.join(img_dir, image_name))

      # Get corresponding mask name from the dictionary
      mask_name = image_mask_dict[image_name]

      # Load mask
      mask = np.load(os.path.join(mask_dir, mask_name))

      images.append(image)
      masks.append(mask)

  # Convert lists to numpy arrays
  images = np.array(images)
  masks = np.array(masks)
  return images, masks

# Function for image data generator
def imageLoader(img_dir, img_list, mask_dir, mask_list, batch_size):

  L = len(img_list)

  # Generator loop (unchanged)
  while True:
    batch_start = 0
    batch_end = batch_size

    while batch_start < L:
      limit = min(batch_end, L)

      # Load images and masks using the defined function
      X, Y = load_img(img_dir, img_list[batch_start:limit], mask_dir, mask_list[batch_start:limit])

      # Shape check (unchanged)
      if X.shape[1:] != (128, 128, 128, 3):
        batch_start += batch_size
        batch_end += batch_size
        continue

      yield (X, Y)  # Yield the batch of images and masks

      batch_start += batch_size
      batch_end += batch_size

In [36]:
train_img_dir = "BraTask_TrainingData/images/"
train_mask_dir = "BraTask_TrainingData/masks/"

val_img_dir = "BraTask_ValidationData/images/"
val_mask_dir = "BraTask_ValidationData/masks/"

train_img_list=os.listdir(train_img_dir)
train_mask_list = os.listdir(train_mask_dir)

val_img_list=os.listdir(val_img_dir)
val_mask_list = os.listdir(val_mask_dir)

In [37]:
batch_size = 2
train_img_datagen = imageLoader(train_img_dir, train_img_list, 
                                train_mask_dir, train_mask_list, batch_size)

val_img_datagen = imageLoader(val_img_dir, val_img_list, 
                                val_mask_dir, val_mask_list, batch_size)



In [38]:
#Define loss, metrics and optimizer to be used for training
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.metrics import MeanIoU
from Unet import simple_unet_model
import evaluation_metrics as em

import tensorflow as tf

# TensorFlow'un GPU kullanımını etkinleştirme
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Modeli oluştur
model = simple_unet_model(128, 128, 128, 3, 4)

num_classes = 4

# Loss fonksiyonu, metrikler ve optimizasyon algoritmasını tanımla
loss_function = categorical_crossentropy
metrics = [MeanIoU(num_classes=num_classes)]
optimizer = Adam(lr=0.001)

# Modeli derle
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001), metrics = ['accuracy',tf.keras.metrics.MeanIoU(num_classes=4),
                                                                                                                 em.dice_coef_edema ,em.dice_coef_enhancing] )



# Veri setini yükle ve uygun hale getir
# Örneğin:
# train_img_datagen, val_img_datagen = ... (veri artırma ve veri seti yükleme işlemleri)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model_{epoch:02d}.h5',  # Kaydedilecek dosya adı, epoch numarasını içerir
    save_freq='epoch')

# Modeli eğit
history = model.fit(train_img_datagen,
                    steps_per_epoch=300,
                    epochs=10,
                    verbose=1,
                    validation_data=val_img_datagen,
                    validation_steps=300
                   ,batch_size=2,callbacks=[checkpoint_callback] )



# Modeli kaydetmek isterseniz
model.save('3d_segmentation_model.h5')

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                28, 3)]                                                           
                                                                                                  
 conv3d_152 (Conv3D)            (None, 128, 128, 12  1312        ['input_9[0][0]']                
                                8, 16)                                                            
                                                                                                  
 dropout_72 (Dropout)           (None, 128, 128, 12  0           ['conv3d_152[0][0]']             
                                8, 16)                                                      

ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/model_8/conv3d_170/Conv3D/Conv3DBackpropInputV2' defined at (most recent call last):
    File "C:\Users\yildi\miniconda3\envs\py309\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\yildi\miniconda3\envs\py309\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\yildi\miniconda3\envs\py309\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\yildi\miniconda3\envs\py309\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\yildi\miniconda3\envs\py309\lib\site-packages\ipykernel\kernelapp.py", line 701, in start
      self.io_loop.start()
    File "C:\Users\yildi\miniconda3\envs\py309\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
 